In [1]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import json
import os
from utils import get_assistant_texts, get_tag_masks, remove_tags, replace_tags, get_tags, get_message_ids, get_val_scores, get_prev_messages
import time
from tqdm import tqdm
import threading
import queue
from itertools import islice

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make natural validations set

In [2]:
import re
user_prefix = "USER:"

def get_statements(text, strip=True, include_prev_messages=False):
    """returns individual statements without context"""
    ast_texts = get_assistant_texts(text)
    prev_messages = get_prev_messages(text)
    pattern = "\[\[\d+\]\]"
    statements = []
    for prev, t in zip(prev_messages, ast_texts):
        sts = re.split(pattern, t)
        if include_prev_messages:
            sts[0] = prev + "\n\nASSISTANT: " + sts[0]
        if strip:
            sts = [s.strip() for s in sts if len(s.strip()) > 2]
        statements.append(sts)
    return statements

def get_full_statements(text, include_prev_messages=True):
    """returns a list of lists of fully contextualized statements in each transcript"""
    statements = get_statements(text, strip=False, include_prev_messages=include_prev_messages)
    prefs = [["".join(sts[:i]) for i in range(1, len(sts) + 1)] for sts in statements]
    return prefs

In [3]:
pathname = "data/natural/oasst_alpaca_labeled.txt"
with open(pathname, "r") as f:
    text = f.read()
val_df = get_val_scores(pathname)
statements_without_context = get_statements(text)
statements = get_full_statements(text)
val_df

,scores
d48005ef42404f53fd1e16814f066e83,"[Certainly True, Certainly False, False, Certa..."
51143b0c-e9eb-445e-a27b-2b85264cb165,"[False, Uncertain, N/A, N/A, N/A]"
8a80c4cb-424c-40ac-bb23-7d473b3b2691,[N/A]
4cbaf274-bfca-44e0-8367-8f4ecbf408a1,[True]
a88a4fc2-2880-4378-a18d-fb76c8cc98f0,"[False, False, Certainly True]"
...,...
6bf2cc3b-c2a9-4915-8eee-fbf0fe58eeb1,"[N/A, True, Certainly False, True, Certainly F..."
6693b4a0-bef9-41c6-8ea5-5c3c3716d50b,"[N/A, True, N/A, N/A, False, True, N/A, N/A, N..."
350a2123-93d9-4994-a9a9-756adad86570,"[Certainly True, Certainly True, Certainly Tru..."
96c6fe02-c3f7-41b1-9e43-d52b717c2d9e,"[N/A, True, Certainly False, False, Certainly ..."


In [4]:
statements[0]

["USER: Who's Marshall Rosenberg and what is his work all about? How important is it?\n\nASSISTANT: Marshall Rosenberg was a Jewish psychologist, author, educator and movement leader in the fields of psychoanalysis, humanistic psychology, interpersonal relations, and nonviolent conflict resolution",
 'USER: Who\'s Marshall Rosenberg and what is his work all about? How important is it?\n\nASSISTANT: Marshall Rosenberg was a Jewish psychologist, author, educator and movement leader in the fields of psychoanalysis, humanistic psychology, interpersonal relations, and nonviolent conflict resolution. He developed a system known as "Rosenbergianism," which focused on personal growth through psychological transformation rather than psychodynamic insight or therapy',
 'USER: Who\'s Marshall Rosenberg and what is his work all about? How important is it?\n\nASSISTANT: Marshall Rosenberg was a Jewish psychologist, author, educator and movement leader in the fields of psychoanalysis, humanistic psy

In [8]:
get_assistant_texts(text)

['Marshall Rosenberg was a Jewish psychologist, author, educator and movement leader in the fields of psychoanalysis, humanistic psychology, interpersonal relations, and nonviolent conflict resolution[[1]]. He developed a system known as "Rosenbergianism," which focused on personal growth through psychological transformation rather than psychodynamic insight or therapy[[2]].\n\nWhile many people have heard of him for his contributions to psychoanalysis and psychology, he is also well-known for his views on social issues such as war, poverty, racism, and environmental destruction[[3]]. His ideas were especially influential during the anti-Vietnam War period and he was an outspoken advocate for peace and justice[[4]].\n\nOverall, Marshall Rosenberg\'s work continues to be highly regarded and relevant today, particularly within the fields of psychology and politics[[5]]. His ideas continue to inspire new generations of psychologists and activists who are committed to improving society thr

In [4]:
ast_texts

['Marshall Rosenberg was a Jewish psychologist, author, educator and movement leader in the fields of psychoanalysis, humanistic psychology, interpersonal relations, and nonviolent conflict resolution[[1]]. He developed a system known as "Rosenbergianism," which focused on personal growth through psychological transformation rather than psychodynamic insight or therapy[[2]].\n\nWhile many people have heard of him for his contributions to psychoanalysis and psychology, he is also well-known for his views on social issues such as war, poverty, racism, and environmental destruction[[3]]. His ideas were especially influential during the anti-Vietnam War period and he was an outspoken advocate for peace and justice[[4]].\n\nOverall, Marshall Rosenberg\'s work continues to be highly regarded and relevant today, particularly within the fields of psychology and politics[[5]]. His ideas continue to inspire new generations of psychologists and activists who are committed to improving society thr

# Make the validation set

In [1]:
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler
from annotation_utils import make_transcript
import pandas as pd

path1 = "data/oasst/generated_responses/50_transcripts_keyfan_bloomz-rlhf(max_length=564;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json"
path2 = "data/oasst/generated_responses/50_transcripts_OpenAssistant_oasst-sft-4-pythia-12b-epoch-3.5(max_length=533;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json"

df1 = pd.read_json(path1).iloc[:50]
df2 = pd.read_json(path2).iloc[-1:]
brennan_df = pd.concat([df1, df2])

apt_tagger = AptTagger(device="cuda:7")

def add_ann_cols(df):
    df["tag_mask"] = apt_tagger.tag(df["assistant_text"])
    df["annotated_assistant_text"] = df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
    df["annotated_transcript"] = df.apply(make_transcript, axis=1, args=(True,))  # True for annotated
    df["templated_annotated_transcript"] = df["annotated_transcript"].apply(lambda x: TruthLabeler.make_input(x)[0])

add_ann_cols(brennan_df)
brennan_df.head()


/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'annotation_utils'

In [ ]:
from hashlib import md5
brennan_df["message_id"] = brennan_df["assistant_text"].apply(lambda x: md5(x.encode()).hexdigest())
brennan_df.head()

,prompt,assistant_text,parent_id,parent_text,prev_messages,role,synthetic,model,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript,message_id
0,A chat between a curious human and an artifici...,"First, try and prioritize your tasks based on ...",f51d172d-4923-4eaf-8883-f0118c898182,can you be more specific? i think a bit of gui...,[I'm completely overwhelmed with all the work ...,assistant,True,keyfan/bloomz-rlhf(max_length=693;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","First, try and prioritize your tasks based on ...",PROMPTER: I'm completely overwhelmed with all ...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,36ccb4cb2be532e9170710eb8819a393
1,A chat between a curious human and an artifici...,"The biggest problem with the Burj Khalifa, my ...",113af868-3868-4276-86dc-dc465d7875b9,What is the biggest problem with the Burj Khal...,[What is the biggest problem with the Burj Kha...,assistant,True,keyfan/bloomz-rlhf(max_length=566;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","The biggest problem with the Burj Khalifa, my ...",PROMPTER: What is the biggest problem with the...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,df951205cc75dbbb35bcbd1314d18911
2,A chat between a curious human and an artifici...,There are many things you can do in Rome that ...,336fb154-f62d-4f01-8fe3-7002b3b49389,I've never been to Rome. What must I see?,[I've never been to Rome. What must I see?],assistant,True,keyfan/bloomz-rlhf(max_length=557;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",There are many things you can do in Rome that ...,PROMPTER: I've never been to Rome. What must I...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,39dcc0f1ed81f493eb2d21e1e6a46137
3,A chat between a curious human and an artifici...,Certainly! Here is what that might look like u...,a2703c6e-0a59-4c54-97b3-50b538502d14,Can you do it again but in Java?,"[Write a simple Hello World program in python,...",assistant,True,keyfan/bloomz-rlhf(max_length=596;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! Here is what that might look like u...,PROMPTER: Write a simple Hello World program i...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,23d7ff066f9971afb04577aca78a35ba
4,A chat between a curious human and an artifici...,Here are some potential ways to encourage more...,9a001945-07a7-4030-8bfd-1994cc42554b,How can I convince someone to be proactive who...,[How can I convince someone to be proactive wh...,assistant,True,keyfan/bloomz-rlhf(max_length=597;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Here are some potential ways to encourage more...,PROMPTER: How can I convince someone to be pro...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,b2306c0c6977ed4b360ed1291b7af51c


In [ ]:
print(brennan_df["templated_annotated_transcript"].iloc[0])

### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
PROMPTER: I'm completely overwhelmed with all the work I need to do. So many people expect stuff form me and I just can't handle it all. The more I work, the less I get done and I have so many unanswered email and things I promised to do, I'm scared to even go look at my inbox. What can I do?

ASSISTANT: The best thing you could do in this situation is to take a day off and relax somewhere, you could read a book or lay in bed watching YouTube or you could talk to someone. But do make sure that you're not missing anything important that has been planned on that day.

PROMPTER: can you be more specific? i think a bit of guidance could help me..

ASSISTANT: First, try and prioritize your tasks based on urgency and importance, then break them down into smaller manageable chunks[[1]]. This will allow you to tackle one task at a time instead of feeling overwhelmed by everything at once[[2]]. Also try and schedule some breaks while working, as lo

In [2]:
import pandas as pd
old_combined_df = pd.read_json("data/label-validation/old_tagging_style/transcripts_combined.json")
old_combined_df.drop(columns=["parent_text", "role", "rank", "user_text"], inplace=True)

add_ann_cols(old_combined_df)
old_combined_df.head()

FileNotFoundError: File data/label-validation/old_tagging_style/transcripts_combined.json does not exist

# Combine val sets

In [1]:
import pandas as pd
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler
from utils import make_transcript

apt_tagger = AptTagger()

def add_ann_cols(df):
    df["tag_mask"] = apt_tagger.tag(df["assistant_text"])
    df["annotated_assistant_text"] = df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
    df["annotated_transcript"] = df.apply(make_transcript, axis=1, args=(True,))  # True for annotated
    df["templated_annotated_transcript"] = df["annotated_transcript"].apply(lambda x: TruthLabeler.make_input(x)[0])

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
paths = [
    "data/generated-responses/rank_0_7400_transcripts_mmlu_meta-llama_Llama-2-7b-chat-hf.json",
    "data/generated-responses/rank_1_7450_transcripts_mmlu_meta-llama_Llama-2-7b-chat-hf.json",
    "data/generated-responses/rank_0_700_transcripts_mmlu_meta-llama_Llama-2-13b-chat-hf.json",
    "data/generated-responses/rank_1_750_transcripts_mmlu_meta-llama_Llama-2-13b-chat-hf.json",
]
n_each = 3
dfs = [pd.read_json(paths[0]).iloc[:n_each], pd.read_json(paths[1]).iloc[:n_each],
       pd.read_json(paths[2]).iloc[n_each:2 * n_each], pd.read_json(paths[3]).iloc[n_each:2 * n_each]]
cat_df = pd.concat(dfs)

add_ann_cols(cat_df)
print(cat_df["templated_annotated_transcript"].iloc[0])

### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
PROMPTER: Answer this question:

What happens to the image formed by a concave mirror as the object is moved from far away to near the focal point?
(A) The image moves away from the mirror and gets shorter.
(B) The image moves away from the mirror and gets taller.
(C) The image moves toward the mirror and gets shorter.
(D) The image moves toward the mirror and gets taller.

ASSISTANT: Hello[[1]]! I'm here to help you with your question[[2]]. However, I must inform you that the statement in the question is incorrect[[3]]. When an object is moved from far away to near the focal point of a concave mirror, the image formed will actually appear larger, not smaller[[4]]. This is because the concave mirror has a negative focal length, which means it diverges light rays and makes objects appear larger than they really are[[5]]. So, the correct answer is (B) The image moves away from the mirror and gets taller.[[6]]
###

First, decide whether each 

In [6]:
cat_df.head()

,message_id,prompt,assistant_text,parent_id,prev_messages,role,model,template_name,gen_kwargs,subject,answer_index,correct_answer,random_incorrect_answer,random_other_subject,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript
0,4584ad83-28df-4c51-8f9f-f533d0649ae0,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Hello! I'm here to help you with your question...,4dceb093b525b4687f5da7a758ed5b85,[Answer this question:\n\nWhat happens to the ...,assistant,meta-llama/Llama-2-7b-chat-hf,vanilla,max_length=4096;temperature=1.0;repetition_pen...,High School Physics,1,The image moves away from the mirror and gets ...,The image moves toward the mirror and gets tal...,Professional Accounting,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Hello[[1]]! I'm here to help you with your que...,PROMPTER: Answer this question:\n\nWhat happen...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
1,bde7de09-80ef-44f5-bdce-48426cbae387,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Of course! I'm here to help you with this ques...,d5f628a32076eb706cc03f6133b70493,[Weber said that the 'spirit of capitalism' co...,assistant,meta-llama/Llama-2-7b-chat-hf,imperative,max_length=4096;temperature=1.0;repetition_pen...,Sociology,3,Calvinists who engaged in ascetic practices to...,new religious movements who rejected tradition...,International Law,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",Of course[[1]]! I'm here to help you with this...,PROMPTER: Weber said that the 'spirit of capit...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
2,f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...","Oh, wow! That's such an awesome question! *exc...",bf8608f71ce32d8d115cfbec5a6f56c9,"[Of the following animals, which has the longe...",assistant,meta-llama/Llama-2-7b-chat-hf,5yo,max_length=4096;temperature=1.0;repetition_pen...,Human Aging,1,Blue whale,Dog,High School Geography,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","Oh, wow[[1]]! That's such an awesome question[...","PROMPTER: Of the following animals, which has ...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
0,c20d1031-c292-4039-89f2-209ca862efc1,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",The best translation into predicate logic for ...,f142a6a4ac2472d72aa90b127c2f4ddc,[Select the best translation into predicate lo...,assistant,meta-llama/Llama-2-7b-chat-hf,answer_only,max_length=4096;temperature=1.0;repetition_pen...,Formal Logic,3,Nd ⊃ (Bj ∨ Sl),Dn ≡ (Bx • Ly),Professional Medicine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The best translation into predicate logic for ...,PROMPTER: Select the best translation into pre...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
1,94ea7b6a-dcbe-4fb4-a94c-5427447cc333,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Thank you for asking! I'm here to help you wit...,b5423fa1a663e0a13ff0770d371b681c,[Answer this question:\n\nCells taken from a b...,assistant,meta-llama/Llama-2-7b-chat-hf,vanilla,max_length=4096;temperature=1.0;repetition_pen...,Human Aging,1,Stop dividing when their telomere strands are ...,Never divide,Clinical Knowledge,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Thank you for asking[[1]]! I'm here to help yo...,PROMPTER: Answer this question:\n\nCells taken...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...


In [7]:
out_str = ""
for row in cat_df.iloc:
    out_str += f"[[MESSAGE_ID]] {row['message_id']}\n"
    out_str += f"[[MODEL_NAME]] {row['model']}\n"
    out_str += f"[[SUBJECT]] {row['subject']}\n"
    out_str += f"[[CORRECT_ANSWER]] {row['correct_answer']}\n"
    out_str += f"[[INPUT_TEXT]] {row['templated_annotated_transcript']}\n"
    out_str += "[[NOT YET ANNOTATED]]"
    out_str += "\n\n\n\n"

out_file = f"data/validation/BLANK_{len(cat_df)}_val_labels_all.txt"
with open(out_file, "w") as f:
    f.write(out_str)

cat_df.to_json(f"data/validation/{len(cat_df)}_transcripts.json", orient="records")

# clean up dataframes

In [33]:
model_name = "keyfan_bloomz-rlhf(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95)"
data_path = f"data/transcripts_{model_name}.csv"
df = pd.read_csv(data_path)

In [34]:
print(f"Total number of transcripts: {len(df)}")
df = df.drop_duplicates(subset=['prompt'])
df = df.reset_index(drop=True)
len(df)

Total number of transcripts: 843


831

In [35]:
df["response"].iloc[0]

"Sure! DHCP (Dynamic Host Configuration Protocol) is a protocol used by network devices such as routers to assign IP addresses to client computers or other networked devices.\n\nDHCP assigns each device on a local area network (LAN) with a unique public address called an Internet Protocol (IP) address. This makes it possible for all networked devices on the LAN to communicate with one another using standard internet protocols like TCP/IP.\n\nBy default, most operating systems come preconfigured with static IP configurations meaning that the assigned IP address remains constant until manually changed through configuration changes in the system settings of your router or modem or even via software installations from certain sources.  When this happens, it's common practice for people still not familiar enough with DHCP to assume that they have manually set up their PCs with dynamic DNS so they are always downloading web applications directly off the internet and thus automatically being 

In [36]:
import uuid
def get_asst_text(text):
    if "smartplat" in data_path:
        bits = text.split("ASSISTANT:")
        bits = bits[1].split("USER:")
        bits = bits[0].split("</s>")  # TODO: figure out why the model is outputting this
        return bits[0].strip()
    elif "keyfan_bloomz" in data_path:
        return text
    else:
        raise Exception("Unknown dataset")

df["assistant_text"] = df["response"].apply(get_asst_text)
df["user_text"] = df["parent_text"]
df["message_id"] = [str(uuid.uuid4()) for _ in range(len(df))]
df["assistant_text"].iloc[0], df["user_text"].iloc[0]

("Sure! DHCP (Dynamic Host Configuration Protocol) is a protocol used by network devices such as routers to assign IP addresses to client computers or other networked devices.\n\nDHCP assigns each device on a local area network (LAN) with a unique public address called an Internet Protocol (IP) address. This makes it possible for all networked devices on the LAN to communicate with one another using standard internet protocols like TCP/IP.\n\nBy default, most operating systems come preconfigured with static IP configurations meaning that the assigned IP address remains constant until manually changed through configuration changes in the system settings of your router or modem or even via software installations from certain sources.  When this happens, it's common practice for people still not familiar enough with DHCP to assume that they have manually set up their PCs with dynamic DNS so they are always downloading web applications directly off the internet and thus automatically being

In [37]:
df.to_json(f"data/transcripts_{model_name}.json", orient="records")

In [38]:
from TruthLabeler import TruthLabeler
from AptTagger import AptTagger
apt_tagger = AptTagger()

In [42]:
df.iloc[16].parent_id

'dfe226cd-1663-45e3-ab80-7b6065da46ac'

In [39]:
raw_df = df.iloc[11:21]
raw_df["tag_mask"] = apt_tagger.tag(raw_df["assistant_text"])
raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
raw_df["annotated_transcript"] = raw_df.apply(lambda x: f"USER: {x['user_text']}\n\nASSISTANT: {x['annotated_assistant_text']}", axis=1)
raw_df["input_text"] = raw_df["annotated_transcript"].apply(lambda x: TruthLabeler.make_input(x)[0])
raw_df.head()

/tmp/ipykernel_195852/416794750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df["tag_mask"] = apt_tagger.tag(raw_df["assistant_text"])
/tmp/ipykernel_195852/416794750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
/tmp/ipykernel_195852/416794750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,prompt,response,parent_id,parent_text,role,synthetic,model,assistant_text,user_text,message_id,tag_mask,annotated_assistant_text,annotated_transcript,input_text
11,A chat between a curious human and an artifici...,"As an artificial intelligence language model, ...",6faa03e5-5f0c-483c-9d22-a15505a2b821,Where was your last vaccation?,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,"As an artificial intelligence language model, ...",Where was your last vaccation?,87cdf227-c048-4ccb-a790-0f44dc5ebb6b,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","As an artificial intelligence language model, ...",USER: Where was your last vaccation?\n\nASSIST...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
12,A chat between a curious human and an artifici...,Joseph Scaliger (1557-1636) was an Italian sch...,2f2e3cfd-857f-41b5-9824-0b62f7dd374c,Describe Joseph Scaliger and his contributions...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Joseph Scaliger (1557-1636) was an Italian sch...,Describe Joseph Scaliger and his contributions...,fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joseph Scaliger (1557-1636) was an Italian sch...,USER: Describe Joseph Scaliger and his contrib...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
13,A chat between a curious human and an artifici...,Certainly! A well-chosen color palette is one ...,89b13be2-c82f-493a-9258-1d690440e451,"I am designing a website, can you give me a co...",assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Certainly! A well-chosen color palette is one ...,"I am designing a website, can you give me a co...",69f27058-6253-42bc-892c-5f8a95870ec1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! A well-chosen color palette is one ...,"USER: I am designing a website, can you give m...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
14,A chat between a curious human and an artifici...,Lena was a happy-go-lucky suburban housewife w...,0cd84750-ccf6-4a8e-89bf-02e17a6d0eca,Write me a story synopsis for a novel where th...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Lena was a happy-go-lucky suburban housewife w...,Write me a story synopsis for a novel where th...,b864688c-62cb-420d-bae1-e81677b4fded,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lena was a happy-go-lucky suburban housewife w...,USER: Write me a story synopsis for a novel wh...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
15,A chat between a curious human and an artifici...,Ser introvertido no significa que no puedas te...,b65b2bfa-1fc6-4448-95a5-cc92544d83cf,¿como consigo una novia siendo introvertido?,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Ser introvertido no significa que no puedas te...,¿como consigo una novia siendo introvertido?,3855e40b-823f-4b80-811e-e0071034c507,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Ser introvertido no significa que no puedas te...,USER: ¿como consigo una novia siendo introvert...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...


In [40]:
out_str = ""
for row in raw_df.iloc:
    out_str += f"[[MESSAGE_ID]] {row['message_id']}\n"
    out_str += f"[[INPUT_TEXT]] {row['input_text']}\n"
    out_str += "[[NOT YET ANNOTATED]]"
    out_str += "\n\n\n\n"

out_file = f"data/BLANK_val_labels_{model_name}.txt"
with open(out_file, "w") as f:
    f.write(out_str)
out_file

'data/BLANK_val_labels_keyfan_bloomz-rlhf(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).txt'

In [5]:
import re
ipath= u"./data/label-validation/backup/BLANK_val_labels_brennan.txt"
matches = re.findall(r'[\u4e00-\u9fff]+', ipath)
matches

[]